In [2]:
import logging
import threading
import time
import concurrent.futures

## Threading with one thread

In [17]:
def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,),daemon = True)
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    x.join()
    logging.info("Main    : all done")

17:49:59: Main    : before creating thread
17:49:59: Main    : before running thread
17:49:59: Thread 1: starting
17:49:59: Main    : wait for the thread to finish
17:50:01: Thread 1: finishing
17:50:01: Main    : all done


## Threading with multiple thread(3)

In [28]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    threads = list()
    for index in range(3):
        logging.info("Main    : create and start thread %d.", index)
        x = threading.Thread(target=thread_function, args=(index,))
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        logging.info("Main    : before joining thread %d.", index)
        thread.join()
        logging.info("Main    : thread %d done", index)

11:10:03: Main    : create and start thread 0.
11:10:03: Thread 0: starting
11:10:03: Main    : create and start thread 1.
11:10:03: Thread 1: starting
11:10:03: Main    : create and start thread 2.
11:10:03: Thread 2: starting
11:10:03: Main    : before joining thread 0.
11:10:05: Thread 0: finishing
11:10:05: Main    : thread 0 done
11:10:05: Main    : before joining thread 1.
11:10:05: Thread 2: finishing
11:10:05: Thread 1: finishing
11:10:05: Main    : thread 1 done
11:10:05: Main    : before joining thread 2.
11:10:05: Main    : thread 2 done


## Threading using ThreadPool 

In [3]:
class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

16:15:23: Testing update. Starting value is 0.
16:15:23: Thread 0: starting update
16:15:23: Thread 1: starting update
16:15:23: Thread 0: finishing update
16:15:23: Thread 1: finishing update
16:15:23: Testing update. Ending value is 1.


## Threading using LOCK
#### In this output you can see Thread 0 acquires the lock and is still holding it when it goes to sleep. Thread 1 then starts and attempts to acquire the same lock. Because Thread 0 is still holding it, Thread 1 has to wait. This is the mutual exclusion that a Lock provides.

In [4]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.locked_update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

16:15:59: Testing update. Starting value is 0.
16:15:59: Thread 0: starting update
16:15:59: Thread 1: starting update
16:15:59: Thread 0: finishing update
16:16:00: Thread 1: finishing update
16:16:00: Testing update. Ending value is 2.


## Deadlock Example
### When the program calls l.acquire() the second time, it hangs waiting for the Lock to be released. In this example, you can fix the deadlock by removing the second call